# Test plane-based registration
Test different plane-based registration methods

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [61]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_10_samples_5.0hz', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_10_samples_5.0hz', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [62]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

In [63]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2])
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [64]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

### Decoupled

In [66]:
from planeslam.registration import decoupled_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
T_abs[:3,3] = drone_positions[0]
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

traj_est = abs_traj_transformations[:,:3,3]

average registration time:  0.002999711036682129


In [67]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
traj_est

In [ ]:
# RMSE error
np.mean(np.linalg.norm(traj_est - drone_positions[1:], axis=1))

Decoupled Gauss Newton

In [68]:
from planeslam.registration import decoupled_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
T_abs[:3,3] = drone_positions[0]
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_GN_register(scans[i], scans[i+1])
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

traj_est = abs_traj_transformations[:,:3,3]

final rotation loss:  1.2128511566156274e-05
final rotation loss:  6.244406598553813e-08
final rotation loss:  4.030343892215543e-06
final rotation loss:  2.7736869405103963e-05
final rotation loss:  1.0842509160849713e-05
final rotation loss:  7.734434643415268e-06
final rotation loss:  3.381577271165651e-05
final rotation loss:  1.3068658469992881e-05
final rotation loss:  7.94933739426015e-07
final rotation loss:  4.493691901621482e-05
average registration time:  0.007801342010498047


In [69]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [70]:
# RMSE error
np.mean(np.linalg.norm(traj_est - drone_positions[1:], axis=1))

1.4367215991563784

In [ ]:
R_hat, t_hat = decoupled_GN_register(scans[11], scans[12])

In [ ]:
R_hat

In [ ]:
t_hat